## <center>Search Engine With Tools and Agents</center>

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai

In [24]:
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv('GROQ_API_KEY')


In [25]:
##  Arxiv --> Research Paper
##  Wikipedia --> Wikipedia Articles
##  Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [26]:
##  Used the in-build tool of Wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [27]:
##  Used the in-build tool of ARXIV
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [28]:
tools = [wiki, arxiv]

## <center>Custom Tools</center>

In [29]:
##  Custom Tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retiever = vectordb.as_retriever()
retiever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x70d2d4116fe0>, search_kwargs={})

In [31]:
from langchain.tools.retriever import create_retriever_tool
retiever_tool = create_retriever_tool(retriever=retiever, name="langsmith-search", description="Search any information about Langsmith")

In [32]:
tools = [wiki, arxiv, retiever_tool]

In [33]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/media/r00t/Neuraligence/Generative AI/LangChain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x70d2d99d67a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

##  <center>Run all These tools with Agents and LLM Models</center>

In [34]:
## Tools, LLM --> AgentExecutor

In [46]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model= "Llama3-8b-8192")

In [47]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [48]:
##  Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [49]:
## Agent Executer 
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [50]:
agent_executer.invoke({"input": "tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'tell me about langsmith',
 'output': 'It appears that LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, iterate on prompts, and ship quickly with confidence.\n\nSome of the key features of LangSmith include:\n\n* Observability: LangSmith allows you to analyze traces, configure metrics, dashboards, and alerts to closely monitor your application.\n* Evaluation: LangSmith provides a way to evaluate your application over production traffic, score performance, and get human feedback on your data.\n* Prompt Engineering: LangSmith provides tools to enable and facilitate prompt engineering, helping you find the perfect prompt for your application.\n\nIt seems that LangSmith is designed to make it easier for developers to build and deploy high-quality LLM applications, and to help them iterate a

In [51]:
agent_executer.invoke({"input": "What is machine Learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks with high accuracy, efficiency and flexibility.

> Finished chain.


{'input': 'What is machine Learning?',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks with high accuracy, efficiency and flexibility.'}

In [53]:
agent_executer.invoke({"input": "What is attention is all you need?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'attention is all you need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,A paper on transformer architecture! That's a great topic.

Here's a direct response:

The paper "Attention Is All You Need" introduced the transformer architecture, which revolutionized the field of natural language processing and beyond. The transformer model relies solely on self-attention mechanisms, eliminating the need for recurrence and convolution in sequential data. This breakthrough has far-reaching implications for machine translation, question answering, and many other applications.

> Finished chain.


{'input': 'What is attention is all you need?',
 'output': 'A paper on transformer architecture! That\'s a great topic.\n\nHere\'s a direct response:\n\nThe paper "Attention Is All You Need" introduced the transformer architecture, which revolutionized the field of natural language processing and beyond. The transformer model relies solely on self-attention mechanisms, eliminating the need for recurrence and convolution in sequential data. This breakthrough has far-reaching implications for machine translation, question answering, and many other applications.'}

In [56]:
agent_executer.invoke({"input": "what's Attention is all you need research paper is about on arxiv?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention is all you need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrThank you for the result! It seems that the tool call id "jyggpe64p" yielded a result. Based on the result, I can see that it's a research paper on arXiv with the title "Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models".

> Finished chain.


{'input': "what's Attention is all you need research paper is about on arxiv?",
 'output': 'Thank you for the result! It seems that the tool call id "jyggpe64p" yielded a result. Based on the result, I can see that it\'s a research paper on arXiv with the title "Attention Is All You Need But You Don\'t Need All Of It For Inference of Large Language Models".'}